Задание первое: решить две системы (одну с “точной” правой частью, вторую с изменённой), найти число обусловленности исходной матрицы, посчитать фактическую погрешность и оценку погрешности (задание - 10.4, варианты - 10.5 по методичке, стр.90). Затем реализовать три метода: Гаусса с выбором главного элемента (по желанию) для решения СЛАУ, метод Жордана для нахождения обратной матрицы и метод LU-разложения для нахождения определителя матрицы (варианты - 11.6 по методичке, стр.94).

## Число обусловленности, фактическая погрешность и оценка погрешности

In [2]:
import numpy as np

In [3]:
# Матрица из 7 варианта  10.5

A = np.array(
    [[-402.9, 200.7],
     [1204.2, -603.6]]
)
b = np.array([
    [200],
    [-600]
])

b_changed = np.array([
    [199],
    [-601]
])

In [4]:
# Число обусловленности

from numpy.linalg import norm

def cond(m: np.array) -> float:
    M_inv = np.linalg.inv(m)
    return norm(M_inv) * norm(m)

cond(A)

1338.0029850746614

In [5]:
# Оценка погрешности

def error_estimation(A, d_A, b, d_b) -> float:
    return ( cond(A) / (1 - cond(A) * norm(d_A) / norm(A))) * (norm(d_b) / norm(b) + norm(d_A) / norm(A))

error_estimation(A, np.zeros_like(A), b, b_changed - b)

2.9918656287245793

In [6]:
# Фактическая погрешность

x = np.linalg.solve(A, b)
x_hat = np.linalg.solve(A, b_changed)

norm(x_hat - x) / norm(x)

1.8943391987709555

## Реализация методов

In [ ]:
# Матрица из 1 варианта 11.6

A = np.array([
    [3.278164, 1.046583, -1.378574],
    [1.046583, 2.975937, 0.934251],
    [-1.378574, 0.934251, 4.836173]
])

b = np.array([
    [-0.527466],
    [2.5268770],
    [5.165441]
])

In [29]:
# Метод Гаусса

from numpy import ndarray

class GaussianElimination:
    def __init__(self, A: ndarray, b: ndarray, eps: float=0.005):
        self.M = np.concatenate((A, b), axis=1)
        self.n = self.M.shape[0]
        self.eps = eps
        self.solutions = None

        self.forward()
        self.backward()

    def forward(self):
        for i in range(self.n):
            self.forward_step(i)

    def forward_step(self, k):
        if self.M[k, k] < self.eps:
            print(f'Во время алгоритма произведено деление '
                  f'на малый ведущий элемент A[{k}, {k}] = {self.M[k, k]}')
        self.M[k] = self.M[k] / self.M[k, k]
        for i in range(k + 1, self.n):
            self.M[i] = self.M[i] - self.M[k] * self.M[i][k]

    def backward(self):
        solutions = np.array([self.M[self.n - 1][self.n]])
        for i in range(self.n - 2, -1, -1):
            x = self.M[i][self.n] - np.dot(solutions, self.M[i, (i+1):self.n])
            solutions = np.insert(solutions, 0, x)
        self.solutions = solutions.T

In [48]:
x = GaussianElimination(A, b).solutions
np.matmul(A, x) - b.T

array([[ 1.11022302e-16, -4.44089210e-16,  0.00000000e+00]])

In [27]:
# Схема жордана

class JordanScheme:
    def __init__(self, A: ndarray):
        self.n = A.shape[0]
        self.M = np.concatenate((A, np.identity(self.n)), axis=1)

        self.forward()
        self.inverse = self.M[:, self.n:]

    def forward(self):
        for i in range(self.n):
            self.forward_step(i)

    def forward_step(self, k):
        self.M[k] = self.M[k] / self.M[k, k]
        for i in range(0, self.n):
            if i == k:
                continue
            self.M[i] = self.M[i] - self.M[k] * self.M[i][k]

In [31]:
np.matmul(A, JordanScheme(A).inverse)

array([[ 1.00000000e+00, -9.97386300e-17, -8.01020919e-20],
       [ 3.34997281e-17,  1.00000000e+00,  5.68909005e-17],
       [-7.48687361e-17, -5.53197618e-17,  1.00000000e+00]])

In [113]:
# Нахождение определителя с помощью LU разложения

class LUDecomposition:
    def __init__(self, A: ndarray):
        self.n = A.shape[0]
        self.A = A
        self.U = np.identity(self.n)
        self.L = np.identity(self.n)

        self.generate_decomposition()

    def generate_decomposition(self):
        for i in range(self.n):
            for j in range(i, self.n):
                self.L[j, i] = self.A[j, i]
                if i > 0:
                    self.L[j, i] -= np.dot(self.L[j, 0:i], self.U[0:i, i])
            for j in range(i, self.n):
                self.U[i, j] = self.A[i, j] / self.L[i, i]
                if i > 0:
                    self.U[i, j] -= np.dot(self.L[i, 0:i], self.U[0:i, j]) / self.L[i, i]

    def determinant(self):
        return np.product(np.diag(self.L))

In [114]:
# Убеждаемся что получается правильное разложение

decomp = LUDecomposition(A)
np.matmul(decomp.L, decomp.U) - A

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -2.22044605e-16,  0.00000000e+00]])

In [117]:
# Проверим правильность нахождения определителя

lib_det = np.linalg.det(A)
our_det = decomp.determinant()

print(f'Сторонняя библиотека: {lib_det} \nНаписанный метод: {our_det}')

Сторонняя библиотека: 30.669790066106653 
Написанный метод: 30.669790066106664
